In [51]:
import pandas as pd
import numpy as np
import os
import pickle
import gc
import xgboost as xgb
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split

max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 320000

out_path = r'.'
df = pd.read_csv(r'data/bio/all.csv',)
df = df[['sentence_id', 'intent', 'language', 'written', 'spoken',
       'type']]
df.sentence_id = df.sentence_id.astype(int)
df

,sentence_id,intent,language,written,spoken,type
0,0,OpenArea,en,show,show,plain
1,0,OpenArea,en,last uploaded,last uploaded,mycloudarea
2,0,OpenArea,en,on,on,plain
3,0,OpenArea,en,mycloud,mycloud,plain
4,1,OpenArea,en,show,show,plain
...,...,...,...,...,...,...
1083571,173204,SwitchOnDevice,en,all,all,plain
1083572,173204,SwitchOnDevice,en,ikea,ikea,smarthomedevicebrand
1083573,173204,SwitchOnDevice,en,in,in,plain
1083574,173204,SwitchOnDevice,en,the,the,plain


In [52]:
from tqdm import tqdm
def make_bio(df):
    t2 = pd.DataFrame()
    id_before = -1
    for ind, row in df.iterrows():
        if row.sentence_id != id_before:
            tmp = row
            tmp['intent'] = 'None'
            tmp['type'] = 'plain'
            tmp['spoken'] = '<sep>'
            tmp['written'] = '<sep>'
            tmp['tag'] = 'O'
            t2 = t2.append(tmp)
            id_before = row.sentence_id
        if row['type'] != 'PLAIN' and row['written'] != row['spoken']:
            for i, word in enumerate(row['spoken'].split()):
                tmp = row
                tmp['spoken'] = word
                if i == 0:
                    tmp['tag'] = 'B'
                else:
                    tmp['tag'] = 'I'
                t2 = t2.append(tmp)
        else:
            for i, word in enumerate(row['spoken'].split()):
                tmp = row
                tmp['spoken'] = word
                tmp['tag'] = 'O'
                t2 = t2.append(tmp)
    return t2

In [53]:
k = pd.DataFrame()
base=1000
for i in tqdm(range(500)):
    begin = i * base
    end = (i + 1) * base
    df_ = make_bio(df.iloc[begin:end])
    k = pd.concat([k, df_])
k = k.reset_index(drop=True)
k

100%|██████████| 500/500 [1:13:43<00:00,  8.85s/it]


,intent,language,sentence_id,spoken,tag,type,written
0,None,en,0.0,<sep>,O,plain,<sep>
1,None,en,0.0,<sep>,O,plain,<sep>
2,OpenArea,en,0.0,last,O,mycloudarea,last uploaded
3,OpenArea,en,0.0,uploaded,O,mycloudarea,last uploaded
4,OpenArea,en,0.0,on,O,plain,on
...,...,...,...,...,...,...,...
777764,RainInfo,en,92688.0,in,O,plain,in
777765,RainInfo,en,92688.0,tremosine,O,localsearchlocation,tremosine
777766,RainInfo,en,92688.0,on,B,localsearchtimestampstartday,on 5.1
777767,RainInfo,en,92688.0,fifth,I,localsearchtimestampstartday,on 5.1


In [54]:
k.tag.value_counts()

O    611878
I    122027
B     43864
Name: tag, dtype: int64

In [57]:
k.iloc[:50]

,intent,language,sentence_id,spoken,tag,type,written
0,None,en,0.0,<sep>,O,plain,<sep>
1,None,en,0.0,<sep>,O,plain,<sep>
2,OpenArea,en,0.0,last,O,mycloudarea,last uploaded
3,OpenArea,en,0.0,uploaded,O,mycloudarea,last uploaded
4,OpenArea,en,0.0,on,O,plain,on
5,OpenArea,en,0.0,mycloud,O,plain,mycloud
6,None,en,1.0,<sep>,O,plain,<sep>
7,None,en,1.0,<sep>,O,plain,<sep>
8,OpenArea,en,1.0,latest,O,mycloudarea,latest uploads
9,OpenArea,en,1.0,uploads,O,mycloudarea,latest uploads


In [58]:
df = k
max_data_size = 500000
pad_size = 2
x_data = []
y_data =  pd.factorize(df['tag'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()
for x in df['spoken'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)

def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i : i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

print('Total number of samples:', len(x_data))
print('Use: ', max_data_size)
#x_data = np.array(x_data)
#y_data = np.array(y_data)

print('x_data sample:')
print(x_data[0])
print('y_data sample:')
print(y_data[0])
print('labels:')
print(labels)

Total number of samples: 500000
Use:  500000
x_data sample:
[ -1   0   0   0   0   0   0   0   0   0   0  -1   0   0   0   0   0   0
   0   0   0   0  -1  60 115 101 112  62   0   0   0   0   0  -1  60 115
 101 112  62   0   0   0   0   0  -1 108  97 115 116   0   0   0   0   0
   0  -1]
y_data sample:
0
labels:
Index(['O', 'B', 'I'], dtype='object')


In [59]:
x_train = x_data
y_train = y_data
gc.collect()

x_train, x_valid, y_train, y_valid= train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
gc.collect()
num_class = len(labels)
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)
watchlist = [(dvalid, 'valid'), (dtrain, 'train')]

param = {'objective':'multi:softmax',
         'eta':'0.3', 'max_depth':10,
         'silent':1, 'nthread':-1,
         'num_class':num_class,
         'eval_metric':'merror'}
model = xgb.train(param, dtrain, 500, watchlist, early_stopping_rounds=20,
                  verbose_eval=10)
gc.collect()

pred = model.predict(dvalid)
pred = [labels[int(x)] for x in pred]
y_valid = [labels[x] for x in y_valid]
x_valid = [ [ chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
x_valid = [''.join(x) for x in x_valid]
x_valid = [re.sub('a+$', '', x) for x in x_valid]

gc.collect()

df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors.csv'), index=False)

model.save_model(os.path.join(out_path, 'xgb_model'))


[0]	valid-merror:0.03140	train-merror:0.03072
Multiple eval metrics have been passed: 'train-merror' will be used for early stopping.

Will train until train-merror hasn't improved in 20 rounds.
[10]	valid-merror:0.01508	train-merror:0.01357
[20]	valid-merror:0.01032	train-merror:0.00767
[30]	valid-merror:0.00658	train-merror:0.00408
[40]	valid-merror:0.00560	train-merror:0.00270
[50]	valid-merror:0.00466	train-merror:0.00166
[60]	valid-merror:0.00404	train-merror:0.00104
[70]	valid-merror:0.00356	train-merror:0.00076
[80]	valid-merror:0.00332	train-merror:0.00065
[90]	valid-merror:0.00314	train-merror:0.00054
[100]	valid-merror:0.00306	train-merror:0.00050
[110]	valid-merror:0.00300	train-merror:0.00048
[120]	valid-merror:0.00300	train-merror:0.00047
[130]	valid-merror:0.00288	train-merror:0.00046
[140]	valid-merror:0.00292	train-merror:0.00046
[150]	valid-merror:0.00290	train-merror:0.00046
[160]	valid-merror:0.00288	train-merror:0.00045
[170]	valid-merror:0.00290	train-merror:0.0004

In [60]:
df_erros


,data,predict,target
113,for       ,B,O
146,and       ,O,I
432,and       ,O,I
539,with      ,O,I
678,bbc       ,I,O
...,...,...,...
47531,bradley   ,O,I
48085,<sep>     ,O,B
48451,barbie    ,O,I
48701,fantastic ,I,O


In [62]:
from nltk import pos_tag
from nltk.tree import Tree
from nltk.chunk import conlltags2tree
tokens = ['In', 'Beirut', ',', 'a', 'string', 'of', 'officials',
          'voiced', 'their', 'anger', ',', 'while', 'at',
          'the', 'United', 'Nations', 'summit', 'in', 'New',
          'York', ',', 'Prime', 'Minister', 'Fouad', 'Siniora',
          'said', 'the', 'Lebanese', 'people', 'are', 'resolute',
          'in', 'preventing', 'such', 'attempts', 'from',
          'destroying', 'their', 'spirit', '.']
tags = ['O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'B',
        'I', 'O', 'B', 'O', 'B', 'I', 'O', 'O',
        'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O']
# tag each token with pos
pos_tags = [pos for token, pos in pos_tag(tokens)]
# convert the BIO / IOB tags to tree
conlltags = [(token, pos, tg) for token, pos, tg in zip(tokens, pos_tags, tags)]
ne_tree = conlltags2tree(conlltags)
# parse the tree to get our original text
original_text = []
for subtree in ne_tree:
    # checking for 'O' tags
    if type(subtree) == Tree:
        original_label = subtree.label()
        original_string = " ".join([token for token, pos in                      subtree.leaves()])
        original_text.append((original_string, original_label))
print(original_text)

ValueError: Bad conll tag 'B'